<a href="https://colab.research.google.com/github/kenwkliu/ideas/blob/master/colab/HKMA-OpenAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Visualize the data from HKMA APIs
# https://apidocs.hkma.gov.hk/

import pandas as pd

# interative map
import folium

# bypass SSL cert
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
  ssl._create_default_https_context = ssl._create_unverified_context

# Google colab interactive table
%load_ext google.colab.data_table   

# a library to convert the Latitude and Longitude from degree format to decimal
!pip install dms2dec
from dms2dec.dms_convert import dms2dec

In [ ]:
# Get the daily exchange rate from HKMA 
url = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/er-eeri-daily'

# Read the json URL output into a dataframe 
df = pd.read_json(url)
print(df)

In [ ]:
# Extract the record result into another dataframe
ccyDf = pd.DataFrame(df.loc['records', 'result'])
ccyDf.set_index('end_of_day', inplace=True)

# Display the major ccy
ccyDf[['usd', 'gbp', 'cad', 'aud', 'cny', 'jpy', 'eur']]

In [ ]:
# sort the date in asc order and plot the graph
ccyDf[['aud']].sort_index().plot(figsize = (12, 8))

In [ ]:
# Browse other HKMA examples: https://apidocs.hkma.gov.hk/apidata/

In [ ]:
# Read HK gov open data: https://data.gov.hk/en/
# 11-a-side Soccer Pitches (Artificial Turf Pitch)
url = 'https://www.lcsd.gov.hk/datagovhk/facility/facility-sp11atp.json'
dfFootball = pd.read_json(url)
dfFootball

In [ ]:
# Create an interactive folium map and zoom to HK
HK_LAT, HK_LONG = 22.3, 114.2
fmap = folium.Map(location=[HK_LAT, HK_LONG], zoom_start=11)

# From each row of the footbook dataframe, pin the location in the map with the name and opening_time by a tooltip 
for row in dfFootball.itertuples():
  tip = row.Name_en + ": " + row.Opening_hours_en
  folium.Marker([dms2dec(row.Latitude), dms2dec(row.Longitude)], tooltip=tip).add_to(fmap)

fmap

In [ ]:
# save the map
fmap.save('football.html')

In [ ]:
# Other examples
# Hospital: http://www.ha.org.hk/opendata/facility-hosp.json
# All other JSON format: https://data.gov.hk/en-datasets/format/json
# All other CSV format: https://data.gov.hk/en-datasets/format/csv
